# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [28]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [30]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,whitehorse,60.7161,-135.0538,18.43,46,75,3.60,CA,1722730567
1,1,waitangi,-43.9535,-176.5597,8.79,88,76,1.34,NZ,1722730688
2,2,port elizabeth,-33.9180,25.5701,20.16,21,0,2.57,ZA,1722730689
3,3,invercargill,-46.4000,168.3500,8.90,74,71,1.79,NZ,1722730690
4,4,bredasdorp,-34.5322,20.0403,7.67,92,89,3.47,ZA,1722730691


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [33]:
%%capture --no-display

# Configure the map plot
humidity_map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    frame_width = 700,
    frame_height = 500  
)

# Display the map
humidity_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [36]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_filtered = city_data_df[(city_data_df['Wind Speed'] < 4.5)
                    & (city_data_df['Max Temp'] > 21) 
                    & (city_data_df['Max Temp'] < 27)
                    & (city_data_df['Cloudiness'] == 0)]
# Drop any rows with null values
city_data_filtered = city_data_filtered.dropna()

# Display sample data
city_data_filtered

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,bethel,41.3712,-73.4140,23.84,95,0,1.54,US,1722730696
12,12,porto-vecchio,41.5910,9.2795,25.74,83,0,1.03,FR,1722730701
14,14,jamestown,42.0970,-79.2353,23.80,80,0,1.54,US,1722730703
48,48,peace river,56.2501,-117.2860,24.17,49,0,3.60,CA,1722730746
82,82,awjilah,29.1081,21.2869,26.53,51,0,4.31,LY,1722730786
92,92,pacific grove,36.6177,-121.9166,26.79,85,0,3.60,US,1722730799
94,94,zharkent,44.1628,80.0000,24.32,33,0,2.21,KZ,1722730802
196,196,pontes e lacerda,-15.2261,-59.3353,26.63,33,0,1.64,BR,1722730925
259,259,remire-montjoly,4.9167,-52.2667,25.02,94,0,1.03,GF,1722731001
266,266,broome,42.2506,-75.8330,23.43,81,0,0.89,US,1722731010


### Step 3: Create a new DataFrame called `hotel_df`.

In [48]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_filtered[["City","Country","Lat","Lng",'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
8,bethel,US,41.3712,-73.4140,95,
12,porto-vecchio,FR,41.5910,9.2795,83,
14,jamestown,US,42.0970,-79.2353,80,
48,peace river,CA,56.2501,-117.2860,49,
82,awjilah,LY,29.1081,21.2869,51,
92,pacific grove,US,36.6177,-121.9166,85,
94,zharkent,KZ,44.1628,80.0000,33,
196,pontes e lacerda,BR,-15.2261,-59.3353,33,
259,remire-montjoly,GF,4.9167,-52.2667,94,
266,broome,US,42.2506,-75.8330,81,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [51]:
# Set parameters to search for a hotel
radius = 10000

params = {
    "categories":'accommodation.hotel',
    "limit":1,
    "apiKey":geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bethel - nearest hotel: Hampton Inn Danbury
porto-vecchio - nearest hotel: No hotel found
jamestown - nearest hotel: DoubleTree Jamestown
peace river - nearest hotel: New Ridge Inn
awjilah - nearest hotel: No hotel found
pacific grove - nearest hotel: Pacific Grove Inn
zharkent - nearest hotel: Hotel (good)
pontes e lacerda - nearest hotel: Hotel Avenida
remire-montjoly - nearest hotel: Complexe Belova
broome - nearest hotel: No hotel found
ribeira brava - nearest hotel: Cheerfulway Bravamar Hotel
koungou - nearest hotel: Hôtel Trévani
goiatuba - nearest hotel: No hotel found
yangshuo - nearest hotel: No hotel found
yelizovo - nearest hotel: Николина Усадьба
ipora - nearest hotel: No hotel found
westport - nearest hotel: Norwalk Inn and Conference Center
entebbe - nearest hotel: Airport View Hotel
barcelos - nearest hotel: Hotel Dom Nuno
al abyar - nearest hotel: No hotel found
huelma - nearest hotel: Sol de Mágina
alexandria - nearest hotel: Smoha Zahran Haus (7t

,City,Country,Lat,Lng,Humidity,Hotel Name
8,bethel,US,41.3712,-73.4140,95,Hampton Inn Danbury
12,porto-vecchio,FR,41.5910,9.2795,83,No hotel found
14,jamestown,US,42.0970,-79.2353,80,DoubleTree Jamestown
48,peace river,CA,56.2501,-117.2860,49,New Ridge Inn
82,awjilah,LY,29.1081,21.2869,51,No hotel found
92,pacific grove,US,36.6177,-121.9166,85,Pacific Grove Inn
94,zharkent,KZ,44.1628,80.0000,33,Hotel (good)
196,pontes e lacerda,BR,-15.2261,-59.3353,33,Hotel Avenida
259,remire-montjoly,GF,4.9167,-52.2667,94,Complexe Belova
266,broome,US,42.2506,-75.8330,81,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [53]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    color = "City",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ['Lng','Lat','City','Humidity','Hotel Name','Country']
)

# Display the map
hotel_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)